In [95]:
import pandas as pd
import datetime
import sklearn
import xgboost
import numpy as np

In [83]:
holidays_df = pd.read_csv('holidays_events.csv')
oil_df = pd.read_csv('oil.csv')
stores_df = pd.read_csv('stores.csv')
test_df = pd.read_csv('test.csv')
train_df = pd.read_csv('train.csv')
transactions_df = pd.read_csv('transactions.csv')

In [84]:
joined_df = train_df.merge(holidays_df, on='date', how='left')
joined_df = joined_df.merge(oil_df, on='date', how='left')
joined_df = joined_df.merge(stores_df, on = 'store_nbr', how='left')
# joined_df = joined_df.merge(transactions_df, on=['store_nbr', 'date'], how='left')
joined_df = joined_df.set_index('id')

In [85]:
joined_test_df = test_df.merge(holidays_df, on='date', how='left')
joined_test_df = joined_test_df.merge(oil_df, on='date', how='left')
joined_test_df = joined_test_df.merge(stores_df, on = 'store_nbr', how='left')
# joined_test_df = joined_test_df.merge(transactions_df, on=['store_nbr', 'date'], how='left')
joined_test_df = joined_test_df.set_index('id')

In [86]:
def map_holidays_test(holiday):
    if not pd.notnull(holiday):
        return 'Work Day'
    return holiday

def map_locale_holidays(locale):
    if not pd.notnull(locale):
        return 'Work Day'
    return locale

In [87]:
joined_df['type_x'] = joined_df['type_x'].map(lambda t: map_holidays_test(t))
joined_df['locale'] = joined_df['locale'].map(lambda t: map_locale_holidays(t))

In [88]:
joined_test_df['type_x'] = joined_test_df['type_x'].map(lambda t: map_holidays_test(t))
joined_test_df['locale'] = joined_test_df['locale'].map(lambda t: map_locale_holidays(t))

In [89]:
dropped_columns = ['locale_name', 'description', 'transferred'] #transferred might be useful 
joined_df = joined_df.drop(columns=dropped_columns)
joined_test_df = joined_test_df.drop(columns=dropped_columns)

In [96]:
def add_temporal_features(df):
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['week'] = df['date'].dt.isocalendar().week
    df['quarter'] = df['date'].dt.quarter
    df['day_of_week'] = df['date'].dt.day_name()
    df['day_of_month'] = df['date'].dt.day
    df['day_of_year'] = df['date'].dt.dayofyear
    df['is_weekend'] = (df['day_of_week'].isin(['Saturday', 'Sunday'])).astype(int)
    df['is_month_start'] = df['date'].dt.is_month_start.astype(int)
    df['is_month_end'] = df['date'].dt.is_month_end.astype(int)
    df['dayofweek'] = df['date'].dt.dayofweek
    
    # Cyclical encoding for temporal features
    df['day_sin'] = np.sin(2 * np.pi * df['day_of_year']/365)
    df['day_cos'] = np.cos(2 * np.pi * df['day_of_year']/365)
    df['month_sin'] = np.sin(2 * np.pi * df['month']/12)
    df['month_cos'] = np.cos(2 * np.pi * df['month']/12)
    
    return df

In [97]:
class DateFormater():
    @staticmethod
    def day(d):
        year, month, day = [int(x) for x in d.split('-')]
        date = datetime.datetime(year, month, day)
        return date.strftime('%a') 

    @staticmethod
    def month(d):
        year, month, day = [int(x) for x in d.split('-')]
        date = datetime.datetime(year, month, day)
        return date.strftime('%B')

In [98]:
# joined_df['day_of_week'] = joined_df['date'].map(lambda d: DateFormater.day(d))
# joined_df['month'] = joined_df['date'].map(lambda d: DateFormater.month(d))

# joined_test_df['day_of_week'] = joined_test_df['date'].map(lambda d: DateFormater.day(d))
# joined_test_df['month'] = joined_test_df['date'].map(lambda d: DateFormater.month(d))

In [99]:
joined_df['date'] = pd.to_datetime(joined_df['date'])
joined_test_df['date'] = pd.to_datetime(joined_test_df['date'])

In [100]:
joined_df = add_temporal_features(joined_df)
joined_test_df = add_temporal_features(joined_test_df)

In [101]:
joined_df['dcoilwtico'] = joined_df['dcoilwtico'].interpolate(method='linear')
joined_test_df['dcoilwtico'] = joined_test_df['dcoilwtico'].interpolate(method='linear')

In [102]:
joined_df['dcoilwtico'].loc[0:joined_df.dcoilwtico.first_valid_index()] = joined_df.loc[joined_df.dcoilwtico.first_valid_index()]['dcoilwtico']
joined_test_df['dcoilwtico'].loc[0:joined_test_df.dcoilwtico.first_valid_index()] = joined_test_df.loc[joined_test_df.dcoilwtico.first_valid_index()]['dcoilwtico']

C:\Users\Fabi\AppData\Local\Temp\ipykernel_10356\1719069923.py:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  joined_df['dcoilwtico'].loc[0:joined_df.dcoilwtico.first_valid_index()] = joined_df.loc[joined_df.dcoilwtico.first_valid_index()]

In [103]:
joined_df = joined_df.drop(columns=['date'])
joined_test_df = joined_test_df.drop(columns=['date'])

In [104]:
y = joined_df['sales']
X = joined_df.drop(columns=['sales'])

In [105]:
X.head()

,store_nbr,family,onpromotion,type_x,locale,dcoilwtico,city,state,type_y,cluster,...,day_of_month,day_of_year,is_weekend,is_month_start,is_month_end,dayofweek,day_sin,day_cos,month_sin,month_cos
id,,,,,,,,,,,,,,,,,,,,,
0,1,AUTOMOTIVE,0,Holiday,National,93.14,Quito,Pichincha,D,13,...,1,1,0,1,0,1,0.017213,0.999852,0.5,0.866025
1,1,BABY CARE,0,Holiday,National,93.14,Quito,Pichincha,D,13,...,1,1,0,1,0,1,0.017213,0.999852,0.5,0.866025
2,1,BEAUTY,0,Holiday,National,93.14,Quito,Pichincha,D,13,...,1,1,0,1,0,1,0.017213,0.999852,0.5,0.866025
3,1,BEVERAGES,0,Holiday,National,93.14,Quito,Pichincha,D,13,...,1,1,0,1,0,1,0.017213,0.999852,0.5,0.866025
4,1,BOOKS,0,Holiday,National,93.14,Quito,Pichincha,D,13,...,1,1,0,1,0,1,0.017213,0.999852,0.5,0.866025


In [106]:
X.count()

store_nbr         3054348
family            3054348
onpromotion       3054348
type_x            3054348
locale            3054348
dcoilwtico        3054348
city              3054348
state             3054348
type_y            3054348
cluster           3054348
year              3054348
month             3054348
week              3054348
quarter           3054348
day_of_week       3054348
day_of_month      3054348
day_of_year       3054348
is_weekend        3054348
is_month_start    3054348
is_month_end      3054348
dayofweek         3054348
day_sin           3054348
day_cos           3054348
month_sin         3054348
month_cos         3054348
dtype: int64

In [115]:
num_columns = ['onpromotion', 'dcoilwtico', 'cluster']
cat_columns = ['family', 'type_x', 'locale', 'city', 'state',
               'type_y']

In [159]:
num_transformer = sklearn.pipeline.Pipeline(steps=[
    ("scaler", sklearn.preprocessing.StandardScaler()),
])

cat_transformer = sklearn.pipeline.Pipeline(steps=[
    ("onehot", sklearn.preprocessing.OneHotEncoder(sparse_output=False))
])

preprocessor = sklearn.compose.ColumnTransformer(transformers=[
    ("num", num_transformer, num_columns),
    ("cat", cat_transformer, cat_columns)
])

pipeline = sklearn.pipeline.Pipeline(steps=[
    ("preprocess", preprocessor),
    # ("regressor", xgboost.XGBRegressor(booster='dart'))
    ("regressor", sklearn.neural_network.MLPRegressor())
])


In [ ]:
pipeline.fit(X, y)

In [129]:
sklearn.model_selection.cross_val_score(pipeline, X, y, scoring='neg_root_mean_squared_error')

array([-467.61623776, -526.20993902, -525.48715348, -559.89138713,
       -540.52844775])

In [131]:
preds = pipeline.predict(joined_test_df)

In [132]:
preds = [max(x, 0) for x in preds]

In [133]:
ans_df = pd.DataFrame({
    'id': [x + 3000888 for x in range(len(preds))],
    'sales' : preds
})

In [134]:
ans_df.set_index('id')

,sales
id,
3000888,0.000000
3000889,0.000000
3000890,9.652735
3000891,2949.263672
3000892,0.000000
...,...
3029395,444.361298
3029396,70.401016
3029397,1266.349487


In [135]:
ans_df.to_csv('ans.csv', index=False)